In [44]:
import getpass
import json
import mlflow
import os
import pandas as pd
import requests
import subprocess
import time

from datetime import datetime
from datetime import timedelta

from domino import Domino

from langchain import LLMMathChain
from langchain.agents import AgentExecutor, AgentType, create_csv_agent, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool, DuckDuckGoSearchRun

from pydantic import BaseModel, Field

In [31]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key: ········


In [32]:
llm = ChatOpenAI(temperature=0)

In [86]:
search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

# create an experiment in MLflow
@tool("create_experiment", return_direct=True)
def create_experiment(experiment_name:str):
    """Create an experiment in Domino"""
    # we'll make the name unique in the project by appending a timestamp so that you and other users can run this cell more than once.
    timestamp = time.time()
    if experiment_name:
        experiment_name = f"{experiment_name}-{timestamp}"
        # below, we'll use the returned experiment_id in calls to mlflow.start_run() to add data to the experiment.
        experiment_id = mlflow.create_experiment(experiment_name.replace("experiment_name = \"", ""))
        print(f"Experiment id: {experiment_id}")
        print(f"Experiment name: {experiment_name}")


# execute a blocking job in Domino
@tool("create_run_job", return_direct=True)
def create_run_job(job_file:str):
    """Create and run  a job in Domino"""
    domino = Domino(
    f"{owner}/{project}",
    api_key=os.environ["DOMINO_USER_API_KEY"],
    host=os.environ["DOMINO_API_HOST"],
    )
    domino.authenticate(domino_token_file = os.getenv('DOMINO_TOKEN_FILE'))
    if job_file:
        # Blocking: this will start the run and wait for the run to finish before returning the status of the run
        domino_run = domino.runs_start_blocking(
            [job_file], title="Started from Domino agent"
        )
    print(domino_run)

# analyze control centre costs
@tool("analyze_costs", return_direct=True)
def analyze_costs(user_query:str) -> str:
    """Answer usage and cost analytics related questions for this deployment"""
    if not user_query:
        return None
    # get all the data and write it to a csv
    csv_file_name = 'control_centre_costs.csv'
    API_KEY = os.environ["DOMINO_USER_API_KEY"]
    URL = "https://johnal33586.marketing-sandbox.domino.tech/v4/gateway/runs/getByBatchId"
    headers = {'X-Domino-Api-Key': API_KEY}
    last_date = datetime.now().strftime('%Y-%m-%d')
    
    last_date = datetime.strftime(
        datetime.strptime(last_date, '%Y-%m-%d') + timedelta(days = 1),
        '%Y-%m-%d',
    )
    
    try:
      os.remove(csv_file_name)
    except:
      pass
    
    batch_ID_param = ""
    while True:
        batch = requests.get(url = URL + batch_ID_param, headers = headers)
        parsed = json.loads(batch.text)
        batch_ID_param = "?batchId=" + parsed['nextBatchId']
        df = pd.DataFrame(parsed['runs'])
        df[df.endTime <= last_date].to_csv(
            csv_file_name,
            mode="a+",
            index=False,
            header=True)
        if len(df.index) < 1000 or len(df.index) > len(df[df.endTime <= last_date].index):
            break
            
    # create a csv agent
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    agent_csv = create_csv_agent(
    llm,
    "control_centre_costs.csv",
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=False
    )
    return agent_csv.run(user_query)

/home/ubuntu/.local/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [87]:
tools = [search_tool, math_tool,create_experiment,create_run_job, analyze_costs]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

In [89]:
agent.run("What is a Domino data set")

'A Domino data set is a read-write folder for storing and sharing data in the Domino Data Lab platform.'

In [90]:
agent.run("create an experiment called agent_exp")

Experiment id: 11
Experiment name: experiment_name = "agent_exp-1704842376.6161933


In [91]:
agent.run("Where is Domino data lab located?")

'Domino Data Lab is located at 135 Townsend St Fl 5, San Francisco, California, 94107.'

In [92]:
agent.run("Run a job using job.py")

{'runId': '659dd4930ba3b8384bafda7d', 'message': 'Run for project integration-test/SearchEngineAgent started. You can view it here:\nhttps://johnal33586.marketing-sandbox.domino.tech/jobs/integration-test/SearchEngineAgent/659dd4930ba3b8384bafda7d\n'}


In [94]:
agent.run("What % of Fortune 100 has Domino Data Lab?")

'Over 20% of the Fortune 100 companies have Domino Data Lab.'

In [96]:
agent.run("What new features were launched in Domino Data Lab, give me a 200 word summary")

'Domino Data Lab has launched powerful new capabilities for building AI, including Generative AI (GenAI), rapidly and safely at scale. They have also introduced an AI Project Hub that streamlines the development process by providing pre-trained foundation models and project templates for 1-click turnkey solutions. Additionally, Domino Data Lab has released updates for accelerating AI time-to-value, reducing costs, implementing responsible AI, and expanding their cloud-based offerings. These new features aim to optimize the application of AI in companies and enable them to leverage its business possibilities.'

In [97]:
agent.run("Which project used the most number of compute hours")

/home/ubuntu/.local/lib/python3.9/site-packages/langchain/agents/agent_toolkits/pandas/base.py:289: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(
/home/ubuntu/.local/lib/python3.9/site-packages/langchain/tools/python/tool.py:140: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


'The project that used the most compute hours is "SearchEngineAgent".'

In [98]:
agent.run("How many hours did the project SearchEngineAgent use on this deployment")

/home/ubuntu/.local/lib/python3.9/site-packages/langchain/agents/agent_toolkits/pandas/base.py:289: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(
/home/ubuntu/.local/lib/python3.9/site-packages/langchain/tools/python/tool.py:140: LangChainPendingDeprecationWarning: On 2023-10-27 this module will be be deprecated from langchain, and will be available from the langchain-experimental package.This code is already available in langchain-experimental.See https://github.com/langchain-ai/langchain/discussions/11680.
  warn_deprecated(


'The usage hours for the project "SearchEngineAgent" on this deployment is approximately 8.82 hours.'